### Статистические отчеты выполнения расписания авиационных перелетов

#### Часть 1.

#### Получить данные из внешнего источника. Аэропорты. 

Внешний источник JSON.

GET http://www.alarstudios.com/test/data.cgi (выдает по 10 строк данных, параметры p=N - номер страницы, code=XXX - код доступа, который необходимо получить с помощью запроса, описанного ниже)

GET https://www.alarstudios.com/test/auth.cgi (выдает результат авторизации и код, параметры username=XXX, password=XXX, откроется на test/123)

Замеченная ошибка: в инструкции для двух запросов указан метод GET, а нужно использовать POST

Получаем авторизационные данные

In [1]:
import requests

auth_url = "https://www.alarstudios.com/test/auth.cgi"
url_data = {
    "username": "test",
    "password": "123"
}
result = requests.post(auth_url, data=url_data)
result = result.json()
if result['status'] != 'ok':
    print('error') 
else:
    access_code = result['code']

Скачиваем данные

In [2]:
from json import JSONDecodeError
request_url = "https://www.alarstudios.com/test/data.cgi"
page = 1
raw_data = []
download = True
url_data = {
    "code": access_code,
    "p": page
}

while download:
    result = requests.post(request_url, data=url_data)
    try:
        result = result.json()
        if result['status'] != 'ok':
            print('error')
            break
        data = result["data"]
        if len(data) < 10:
            download = False
            
        raw_data.extend(data)
    except JSONDecodeError:
        pass
    url_data["p"] += 1

Формируем датафрейм из скачанных данных

In [10]:
import pandas as pd

df = pd.DataFrame(raw_data)
df.to_csv("raw_data.csv")
df.head(10)

,id,name,country,lat,lon
0,KVQQ,CECIL,United States of America,30.218778,-81.877167
1,KNQX,KEY WEST NAS/BOCA CHICA FIELD,United States of America,24.574636,-81.686644
2,KEYW,KEY WEST INTL,United States of America,24.556111,-81.759556
3,KISM,KISSIMMEE GATEWAY,United States of America,28.289806,-81.437083
4,KLAL,LAKELAND LINDER RGNL,United States of America,27.988917,-82.018556
5,KLEE,LEESBURG INTL,United States of America,28.822889,-81.808417
6,KMCF,MACDILL AFB,United States of America,27.849333,-82.521167
7,KMTH,THE FLORIDA KEYS MARATHON,United States of America,24.726194,-81.051361
8,KNRB,MAYPORT NS (ADM DAVID L. MCDONALD FIELD),United States of America,30.391375,-81.424519
9,KMLB,MELBOURNE INTL,United States of America,28.102750,-80.645250


Пыталась найти обознаечние аэропорта Косово, но в загруженных данных его не обнаружила. Вероятно, еще одна ошибка.

In [39]:
# https://en.wikipedia.org/wiki/List_of_airports_in_Kosovo
df[(df['id'] == 'BKPR') | (df['id'] == 'PRN')]

,id,name,country,lat,lon


Оставляем только европейские аэропорты, по ТЗ начинаются с E и L

In [40]:
airports = df[df['id'].str.startswith('E') | df['id'].str.startswith('L')]
airports.head(20)
airports.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 788 entries, 924 to 2228
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   id       788 non-null    object 
 1   name     788 non-null    object 
 2   country  788 non-null    object 
 3   lat      788 non-null    float64
 4   lon      788 non-null    float64
dtypes: float64(2), object(3)
memory usage: 36.9+ KB


#### 2. Получить данные из задания. Типы самолетов

Формат: код, скорость в узлах.

B733 430

PAY2 230

C500 250

B738 440

PA34 130

A320 420

Вручную создаем датафрейм с типами самолетов и их скоростями

In [7]:
airplanes = pd.DataFrame({'code': ['B733', 'PAY2', 'C500', 'B738', 'PA34', 'A320'],
                          'speed': [430, 230, 250, 440, 130, 420]})


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   code    6 non-null      object
 1   speed   6 non-null      int64 
dtypes: int64(1), object(1)
memory usage: 224.0+ bytes


#### 3. Генерация данных для обработки. Расписание

Набор полей:

1. [x] flight number (4 цифры, именно так, это не цифра (123, 12), это 4 отдельные цифры, их всегда 4, например 0000, код оператора нас тут не волнует)
2. [x] origin airport code (из первого шага)
3. [x] destination airport code (из первого шага)
4. [x] aircraft type (из второго шага)
5. [x] weekly schedule (строка всегда из 7-и символов, которая содержит дни недели, по которым летает самолетик, каждый день недели отмечен одной буквой на своем месте: М------ - означает, что самолет летает только по понедельникам, M-W-F-- - по понедельникам, средам и пятницам, MTWT--- - тут буквы повторились, но я думаю, идея ясна, что важно, - это день ВЫЛЕТА)
6. [x] off-block time (UTC и только время, не дата или день!)
7. [x] on-block time (UTC и только время, не дата или день!)

In [122]:
from math import sin, cos, sqrt, atan2, radians
from itertools import combinations_with_replacement, permutations
from random import choice, randint

# Функция для рандомной генерации расписания
def get_schedule():
    return [
        choice("-M"),
        choice("-T"),
        choice("-W"),
        choice("-T"),
        choice("-F"),
        choice("-S"),
        choice("-S"),
    ]

# Функция для расчета расстояния между координатами на карте
def get_distance(dist1, dist2):
    # approximate radius of earth in km
    R = 6373.0

    lat1, lon1 = dist1
    lat2, lon2 = dist2

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

# Функция расчета времени полета
def get_flight_time(air_type, dist1, dist2):
    distance = get_distance(dist1, dist2)
    speed = airplanes[airplanes["code"] == air_type]["speed"].values[0]
    # узлы в километры
    speed = speed * 1.852
    
    return distance / speed


In [133]:
unique_codes = airports['id'].unique()
flight_numbers = list(permutations("0123456789", 4))
dtrange = pd.date_range(start='1/1/2018 17:00:00', end='1/1/2018 23:00:00', freq="30T").values
all_flights = []

for row in airports.iterrows():
    # данные по аэропорту
    flights = []
    
    # Устанавливаем время старта, дата не важна
    time_start = datetime.strptime("01.01.2022 17:00", "%d.%m.%Y %H:%M")
    
    # Генерация рандомного кол-ва полетов для каждого аэропорта
    for i in range(randint(3, 7)):
        
        flight_no = "".join(flight_numbers.pop())
        origin_airp_code = row[1]["id"]
        dest_airp_code = origin_airp_code

        # Защита от рандомной выборки, где destination == origin
        while dest_airp_code == origin_airp_code:
            dest_airp_code = choice(unique_codes)
            
        aircraft = choice(airplanes["code"].values)
        schedule = "".join(get_schedule())
        
        # Обязательно один полет в 23:00
        if i == 0:
            off_time = datetime.strptime("01.01.2022 23:00", "%d.%m.%Y %H:%M")
        else:
            off_time = pd.to_datetime(choice(dtrange))
        
        destination_coords = airports[airports["id"] == dest_airp_code][["lat", "lon"]]
        dest1 = (row[1]["lat"], row[1]["lon"])
        dest2 = (destination_coords["lat"].values[0], destination_coords["lon"].values[0])
        time_in_flight = get_flight_time(aircraft, dest1, dest2)
        on_time = (off_time + timedelta(hours=time_in_flight))
        
        on_time = on_time.strftime("%H:%M")
        off_time = off_time.strftime("%H:%M")
        
        # Добавляем информацию в расписание
        flights.append(
            {
                "flight_number": flight_no,
                "origin_airport_code": origin_airp_code,
                "dest_airport_code": dest_airp_code,
                "aircraft_type": aircraft,
                "weekly_schedule": schedule,
                "off_block_time": off_time,
                "on_block_time": on_time,
            }
        )

    all_flights.extend(flights)


Создаем итоговый датафрейм с расписанием

In [134]:
generated_flights = pd.DataFrame(all_flights)
generated_flights.to_csv("generated_flights.csv")

Условия генерации:
1. [x] Только аэропорты в Европе (код начинается с E или L, плюс Косово)
2. [x] Для каждого аэропорта нужно создать от 3 до 7 (сколько именно - рандом) рейсов в любые другие аэропорты (с тем же ограничением по региону), пары могут повторяться
3. [x] Каждый рейс должен иметь уникальный номер (метод генерации не важен, хоть по порядку)
4. [x] Каким типом самолета летим - рандом (из списка)
5. [x] Каждый рейс должен летать в определенные дни недели (в какие именно - рандом, хоть раз в неделю, хоть каждый день или посередине, но разброс должен быть)
6. [x] Время вылета - рандом, условия: между 1700 и 2300 местого времени (хватает данных для этого?), между вылетами разных рейсов не меньше 30 минут, как минимум один летит после 2230
7. [x] Время прилета нужно рассчитать используя скорость типа и кратчайшее расстояние между аэропортами (данные для этого у вас есть)
